Connect to google drive

In [71]:
from google.colab import drive
drive.mount("/content/drive/", force_remount=True)

Mounted at /content/drive/


Read input video (Change **FILEPATH** to the video you are going to use)

In [0]:
import json
import pandas as pd
import numpy as np
import os, os.path
import time
import csv

N_FACE = 70
N_POSE = 25
N_HAND = 21
FOLDER = '/content/drive/My Drive/636/data/json/'
CSV_FOLDER = '/content/drive/My Drive/636/dataset/'

drop_cols = set(['p_' + str(i) for i in range(N_POSE * 2)])

#DELETE POSE

In [0]:
dataset = pd.read_csv(CSV_FOLDER + 'dataset.csv')

In [0]:
dataset.drop(columns = drop_cols, inplace = True)

In [79]:
dataset.columns[(dataset == 0).all()]

Index([], dtype='object')

In [80]:
dataset.shape

(149102, 227)

In [0]:
number0 = (dataset == 0).astype(int).sum(axis=1)

In [0]:
index_list = []
for i in range(len(number0)):
  if number0[i] >= 84:
    index_list.append(i)

In [0]:
drop_vid = set(dataset.vid[index_list].unique())
dataset.drop(dataset.index[dataset['vid'].isin(drop_vid)],inplace=True)

In [86]:
dataset.shape

(58361, 227)

In [87]:
dataset.index[dataset['label'] == 1].shape[0] / dataset.index[dataset['label'] == 0].shape[0]

2.858833641893679

In [88]:
print(dataset.index[dataset['label'] == 1].shape[0] - dataset.index[dataset['label'] == 0].shape[0])

28113


In [0]:
dataset.reset_index(drop=True, inplace=True)

#DELETE HAND


In [0]:
dataset_re = pd.read_csv(CSV_FOLDER + 'dataset_re.csv')
dataset_re.drop(columns = drop_cols_hand, inplace = True)

In [0]:
dataset_re.columns[(dataset_re == 0).all()]

Index([], dtype='object')

In [0]:
dataset_re.shape

(116607, 53)

In [0]:
number0 = (dataset_re == 0).astype(int).sum(axis=1)
index_list = []
for i in range(len(number0)):
  if number0[i] >= 38:
    index_list.append(i)
drop_vid = set(dataset_re.vid[index_list].unique())
dataset_re.drop(dataset_re.index[dataset_re['vid'].isin(drop_vid)],inplace=True)

In [0]:
dataset_re.shape

(107694, 53)

In [0]:
dataset_re.index[dataset_re['label'] == 1].shape[0] / dataset_re.index[dataset_re['label'] == 0].shape[0]

1.6165994460372224

In [0]:
dataset_re.columns[(dataset_re == 0).all()]

Index([], dtype='object')

In [0]:
print(dataset_re.index[dataset_re['label'] == 1].shape[0], dataset_re.index[dataset_re['label'] == 0].shape[0])

66536 41158


#Balance Dataset

In [0]:
dataset.vid.unique()

In [121]:
drop_list = ['a119-0' + str(i) + 'C' for i in range(426, 650, 2)] + ['a119-0' + str(i) + 'C' for i in range(211, 800, 2)]
# drop_list += ['a119-0' + str(i) + 'C' for i in range(425, 663, 2)]
# drop_list = ['a119-0' + str(i) + 'C' for i in range(110, 831, 5)]
drop_data = dataset.loc[dataset['vid'].isin(drop_list)]
print(drop_data.shape)

(26686, 227)


In [0]:
dataset.drop(dataset.index[dataset['vid'].isin(drop_list)], inplace = True)
dataset.reset_index(drop=True, inplace=True)

In [123]:
print(dataset.index[dataset['label'] == 0].shape[0] / dataset.index[dataset['label'] == 1].shape[0])
print(dataset.index[dataset['label'] == 1].shape[0], dataset.index[dataset['label'] == 0].shape[0])

0.9137816446136185
16551 15124


In [0]:
pos_vid = dataset.vid[dataset['label'] == 1].unique()

In [125]:
from sklearn.model_selection import train_test_split
keep_vid, drop_vid= train_test_split(pos_vid, test_size=0.23, random_state=99)
print(keep_vid.shape, drop_vid.shape)

(72,) (22,)


In [126]:
dataset.index[dataset['vid'].isin(drop_vid)].shape

(3665,)

In [0]:
!rm '/content/drive/My Drive/636/dataset/dataset_face_hand.csv'

In [0]:
dataset.to_csv(CSV_FOLDER + 'dataset_face_hand.csv', index=False)

In [0]:
# for i in zero_cols:
#   print(i)

#Split train and test

In [176]:
vid = dataset.vid.unique()
from sklearn.model_selection import train_test_split
X_train, X_test= train_test_split(vid, test_size=0.15, random_state=15)
print(X_train.shape, X_test.shape)

(171,) (31,)


In [0]:
X_test_data = dataset.loc[dataset['vid'].isin(X_test)]
X_test_data = X_test_data.reset_index()
X_test_data.drop(X_test_data.columns[[0]], axis=1, inplace=True)

In [178]:
X_test_data.shape

(4839, 227)

In [179]:
X_test_data.shape[0] / dataset.shape[0]

0.15277032359905288

In [180]:
X_test_data.index[X_test_data['label'] == 1].shape[0] / X_test_data.index[X_test_data['label'] == 0].shape[0]

1.0204592901878915